## Memo
* Install kaggle-cli and download dataset
* move all `cat*` to `cats/`, same for dogs
* take 1000 random pictures and move them to valid/cats/ with `shuf -n 1000 -e train/cat* | xargs -i mv {} valid/cats/`, same for dogs
Furthermore use
* `ls /train/cats/ | wc -l` to count files in `/train/cats/`
* `ls /train/cats/ | grep -v 'cat'` to find all files that DON'T have cat in it. `-v`-flag inverts search.
* use: `mv /train/cat* /valid/cats` and `cp` to do the rest
* copy utils etc: `cp -t ~/fastai/ vgg16.py vgg16bn.py utils.py resnet50.py`

In [1]:
%matplotlib inline
# path = "data/dogscats/"
path = "data/dogscats/sample/"

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import csv
import math

import utils; reload(utils)
from utils import plots
import vgg16; reload(vgg16)
from vgg16 import Vgg16

batch_size = 8

vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
test_batches = vgg.get_batches(path+'temptest', batch_size=batch_size, shuffle=False)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=2)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Found 12 images belonging to 1 classes.
Epoch 1/2
16/16 [==============================] - 9s - loss: 1.3163 - acc: 0.5625 - val_loss: 0.3623 - val_acc: 0.7500
Epoch 2/2
16/16 [==============================] - 9s - loss: 0.2680 - acc: 0.9375 - val_loss: 0.2652 - val_acc: 0.7500


In [4]:
def batch_dogness(test_batches):
    imgs,nolabel = next(test_batches)
    print(test_batches.filenames)
    preds, idxs, labels = vgg.predict(imgs)
    dogness = np.multiply(preds, idxs) + np.multiply(1.-preds, 1-idxs)
    return dogness

In [5]:
def predict_order_testset(testfolder):
    files_names = os.listdir(path+testfolder)
    files_names = [int(files_names[i][:-4]) for i in range(len(files_names))]
    N = int(math.ceil(len(files_names)/batch_size))
    predictions = [x for i in range(N) for x in batch_dogness(test_batches).tolist()]
    predictions_ordered = [x for (y,x) in sorted(zip(files_names,predictions))]
    return predictions_ordered

In [6]:
def write_csv(predictions_ordered):
    with open('test.csv', 'wb') as f:
        fieldnames = ['id', 'label']
        wr = csv.writer(f, delimiter=',')
        wr.writerow(fieldnames)
        for i in range(len(predictions_ordered)):
            wr.writerow([i+1,predictions_ordered[i]])

In [7]:
pre
write_csv(predictions)

['test/11.jpg', 'test/5.jpg', 'test/7.jpg', 'test/9.jpg', 'test/12.jpg', 'test/6.jpg', 'test/1.jpg', 'test/4.jpg', 'test/8.jpg', 'test/3.jpg', 'test/10.jpg', 'test/2.jpg']
['test/11.jpg', 'test/5.jpg', 'test/7.jpg', 'test/9.jpg', 'test/12.jpg', 'test/6.jpg', 'test/1.jpg', 'test/4.jpg', 'test/8.jpg', 'test/3.jpg', 'test/10.jpg', 'test/2.jpg']
['test/11.jpg', 'test/5.jpg', 'test/7.jpg', 'test/9.jpg', 'test/12.jpg', 'test/6.jpg', 'test/1.jpg', 'test/4.jpg', 'test/8.jpg', 'test/3.jpg', 'test/10.jpg', 'test/2.jpg']
['test/11.jpg', 'test/5.jpg', 'test/7.jpg', 'test/9.jpg', 'test/12.jpg', 'test/6.jpg', 'test/1.jpg', 'test/4.jpg', 'test/8.jpg', 'test/3.jpg', 'test/10.jpg', 'test/2.jpg']
